# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Training Data & Feature views</span>

<span style="font-width:bold; font-size: 1.4rem;">This is the second part of the quick start series of tutorials about Hopsworks Feature Store. This notebook explains how to read from a feature group and create training dataset within the feature store</span>

## **🗒️ In this notebook we will see how to create a training dataset from the feature groups:** 
1. **Select the features** we want to train our model on,
2. **How the features should be preprocessed,**
3. **Create a dataset split** for training and validation data.

![tutorial-flow](images/02_training-dataset.png) 

In [1]:
import hsfs

conn = hsfs.connection(host="[UUID].cloud.hopsworks.ai",
                       project="dev",
                       api_key_value="")
fs = conn.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;"> 🔪 Feature Selection </span>

We start by selecting all the features we want to include for model training/inference.

In [2]:
# Load feature groups.
trans_fg = fs.get_feature_group('transactions', version=1)
window_aggs_fg = fs.get_feature_group('transactions_4h_aggs', version=1)

In [3]:
# Select features for training data.
ds_query = trans_fg.select(["fraud_label", "category", "amount", "age_at_transaction", "days_until_card_expires", "loc_delta"])\
    .join(window_aggs_fg.select_except(["cc_num"]))

ds_query.show(5)

,fraud_label,category,amount,age_at_transaction,days_until_card_expires,loc_delta,trans_volume_mstd,trans_volume_mavg,trans_freq,loc_delta_mavg
0,0,Grocery,93.51,25.334094,175.912280,0.000000,93.510,93.510,93.510,0.000000
1,0,Domestic Transport,65.14,25.335632,175.350486,0.319574,65.140,65.140,65.140,0.319574
2,0,Grocery,0.26,25.336235,175.130347,0.314148,0.260,0.260,0.260,0.314148
3,0,Grocery,1.43,25.336660,174.975058,0.000000,0.845,0.845,0.845,0.157074
4,0,Grocery,19.75,25.344710,172.034664,0.105313,19.750,19.750,19.750,0.105313


Recall that we computed the features in `transactions_4h_aggs` using 4-hour aggregates. If we had created multiple feature groups with identical schema for different window lengths, and wanted to include them in the join we would need to include a prefix argument in the join to avoid feature name clash. See the [documentation](https://docs.hopsworks.ai/feature-store-api/latest/generated/api/query_api/#join) for more details.

🤖 Transformation Functions </span>

We will preprocess our data using *min-max scaling* on numerical features and *label encoding* on categorical features. To do this we simply define a mapping between our features and transformation functions. This ensures that transformation functions such as *min-max scaling* are fitted only on the training data (and not the validation/test data), which ensures that there is no data leakage.

In [4]:
# Load transformation functions.
min_max_scaler = fs.get_transformation_function(name="min_max_scaler")
label_encoder = fs.get_transformation_function(name="label_encoder")

# Map features to transformations.
transformation_functions = {
    "category": label_encoder,
    "amount": min_max_scaler,
    "trans_volume_mavg": min_max_scaler,
    "trans_volume_mstd": min_max_scaler,
    "trans_freq": min_max_scaler,
    "loc_delta": min_max_scaler,
    "loc_delta_mavg": min_max_scaler,
    "age_at_transaction": min_max_scaler,
    "days_until_card_expires": min_max_scaler,
}

## <span style="color:#ff5f27;"> ⚙️ Feature View Creation </span>

The Feature Views allows schema in form of a query with filters, define a model target feature/label and additional transformation functions.
In order to create a Feature View we may use `fs.create_feature_view()`

In [5]:
feature_view = fs.create_feature_view(
    name='transactions_view',
    query=ds_query,
    labels=["fraud_label"],
    transformation_functions=transformation_functions
)

Feature view created successfully, explore it at https://c4bae5a0-ec7e-11ec-9f9d-134c26953371.cloud.hopsworks.ai:443/p/1143/fs/1091/fv/transactions_view/version/1


## <span style="color:#ff5f27;"> 🏋️ Training Dataset Creation</span>

In Hopsworks training data is a query where the projection (set of features) is determined by the parent FeatureView with an optional snapshot on disk of the data returned by the query.

**Training Dataset  may contain splits such as:** 
* Training set - the subset of training data used to train a model.
* Validation set - the subset of training data used to evaluate hparams when training a model
* Test set - the holdout subset of training data used to evaluate a mode

Training dataset is created using `fs.create_train_validation_test_splits()` method.

In [6]:
td_version, td_job = feature_view.create_train_validation_test_splits(
    description = 'transactions_dataset',
    data_format = 'csv',
    val_size = 0.2,
    test_size = 0.1,
    write_options = {'wait_for_job': True},
    coalesce = True,
)

Training dataset job started successfully, you can follow the progress at https://c4bae5a0-ec7e-11ec-9f9d-134c26953371.cloud.hopsworks.ai/p/1143/jobs/named/transactions_view_1_1_create_fv_td_16062022215538/executions


## <span style="color:#ff5f27;"> 🪝 Training Dataset retreival </span>

To retrieve training data from storage (already materialised) or from feature groups direcly we can use `get_training_dataset` or `get_train_validation_test_splits` methods. You need to provide the version of the training dataset you want to retrieve.

In [7]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_view.get_train_validation_test_splits(td_version)

In [8]:
X_train

,category,amount,age_at_transaction,days_until_card_expires,loc_delta,trans_volume_mstd,trans_volume_mavg,trans_freq,loc_delta_mavg
0,0,0.000000e+00,0.010738,0.850105,0.026437,0.000000e+00,0.000000e+00,0.000000e+00,0.026888
1,0,0.000000e+00,0.340523,0.206630,0.224487,0.000000e+00,0.000000e+00,0.000000e+00,0.228318
2,0,0.000000e+00,0.954656,0.874544,0.194834,0.000000e+00,0.000000e+00,0.000000e+00,0.198159
3,0,3.336858e-07,0.374494,0.457623,0.260845,3.336858e-07,3.336858e-07,3.336858e-07,0.265296
4,0,3.336858e-07,0.573886,0.203232,0.024463,1.330339e-01,1.330339e-01,1.330339e-01,0.019678
...,...,...,...,...,...,...,...,...,...
74012,4,3.310497e-03,0.209262,0.547633,0.198189,2.202215e-03,2.202215e-03,2.202215e-03,0.115563
74013,5,4.983598e-03,0.206518,0.227145,0.099138,1.922489e-03,1.922489e-03,1.922489e-03,0.100203
74014,5,7.304049e-03,0.909320,0.728360,0.227438,4.702467e-03,4.702467e-03,4.702467e-03,0.204941
74015,8,1.698461e-04,0.132060,0.816005,0.230563,1.698461e-04,1.698461e-04,1.698461e-04,0.234498


In [9]:
X_val

,category,amount,age_at_transaction,days_until_card_expires,loc_delta,trans_volume_mstd,trans_volume_mavg,trans_freq,loc_delta_mavg
0,0,0.000000e+00,0.047263,0.943591,0.037840,0.000000e+00,0.000000e+00,0.000000e+00,0.038485
1,0,0.000000e+00,0.063646,0.130012,0.000046,0.000000e+00,0.000000e+00,0.000000e+00,0.000047
2,0,3.336858e-07,0.817793,0.367259,0.124282,3.336858e-07,3.336858e-07,3.336858e-07,0.126403
3,0,6.673716e-07,0.030149,0.436620,0.219298,6.673716e-07,6.673716e-07,6.673716e-07,0.223041
4,0,6.673716e-07,0.042524,0.199982,0.193627,6.673716e-07,6.673716e-07,6.673716e-07,0.196931
...,...,...,...,...,...,...,...,...,...
21256,4,3.275126e-03,0.209262,0.547634,0.060302,3.275126e-03,3.275126e-03,3.275126e-03,0.061331
21257,5,1.042768e-03,0.499534,0.472921,0.082375,1.042768e-03,1.042768e-03,1.042768e-03,0.083781
21258,5,4.428011e-03,0.922000,0.285555,0.099245,2.328126e-03,2.328126e-03,2.328126e-03,0.073158
21259,5,5.702691e-03,0.206518,0.227144,0.103895,2.155544e-03,2.155544e-03,2.155544e-03,0.101301


The feature view and training dataset are now visible in the UI

![fg-overview](images/fv_overview.gif)

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03 </span>

In the following notebook, we will train a model on the dataset we created in this notebook and have quick overview of the lineage.